## 1. Podstawy - propagacja wsteczna, metoda gradientu

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Stwórzmy jakiś zbiór, którego nie da się łatwo "uchwycić" metodami liniowymi. Najprostszym pomysłem są okręgi o różnych promieniach, i klasie zależnej od promienia.

In [ ]:
n_classes = 3
points_per_class = 200
X = np.zeros((points_per_class*n_classes, 2))
Y = np.zeros(points_per_class*n_classes, dtype=int)
for i in range(n_classes):
    idx = range(points_per_class*i,points_per_class*(i+1))
    
    r = i + np.random.random(points_per_class)* 0.5 # 
    t = np.random.random(points_per_class) *2* np.pi
    
    X[idx] = np.c_[r*np.sin(t), r*np.cos(t)]
    Y[idx] = int(i)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=Y, s=40, cmap=plt.cm.Spectral)
plt.show()

Pierwszą próbę klasyfikacji zrobimy za pomocą prostego klasyfikatora liniowego, to znaczy bezpośrednio po warstwie wejściowej mamy 3-elementową warstwę wyjściową reprezentującą prawdopobieństwo przynależności do danej klasy.

Postać macierzowa:
$$ y = f(U),$$
$$ U = XW +b $$

gdzie $f$ jest funkcją softmax:,

$$ \hat{y_j} = \sigma (u)_j = \frac{\exp{u_j}}{\sum_{j = 1}^{k} \exp{u_j}} $$

dla $j = 1, ..., k$.

Będziemy optymalizować funkcję straty 
$$ L(y, \hat{y}) = -\frac{1}{N} \sum_{n} y_n\log{\hat{y_n}}  $$

Przydatna rzecz: 
$$ \frac{\partial L}{\partial u_j} = \hat{y_j} - y_j $$

In [ ]:
# randomizacja wartości początkowych
D = 2
K = 3
W = 1 * np.random.randn(D,K)
b = np.zeros((1,K))

# parametry 
step_size = 1e-0
reg = 1e-2

num_examples = X.shape[0]

for i in range(200):  
    # obliczamy scory oraz przynależności do klas przy aktualnych wagach
    scores = np.dot(X, W) + b 
    
    exp_scores = np.exp(scores)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True) # [N x K]

    # obliczamy funkcję straty dla aktualnej predykcji
    corect_logprobs = -np.log(probs[range(num_examples),Y])
    data_loss = np.sum(corect_logprobs)/num_examples
    reg_loss = 0.5*reg*np.sum(W*W)
    loss = data_loss + reg_loss
    if i % 20 == 0:
        print("iteration %d: loss %f" % (i, loss))

    # wyliczamy gradient funkcji straty
    dscores = probs
    dscores[range(num_examples),Y] -= 1 # globalnie odejmujemy od obliczonych 
                                        # prawdopodobieństw wartość y_j, aby otrzymać gradient funkcji straty
    dscores /= num_examples
    
    # it's backpropagation time!
    dW = np.dot(X.T, dscores)
    db = np.sum(dscores, axis=0, keepdims=True)

    # dodajemy jeszcze wpływ regularyzacji do gradientu wag
    dW += reg * W

    # aktualizujemy parametry
    W += -step_size * dW
    b += -step_size * db

In [ ]:
# predykcja - postępujemy tak, jak na początku kroku uczenia
scores = np.dot(X, W) + b
predicted_class = np.argmax(scores, axis=1) # nie musimy wyliczać konkretnych prawdopodobienstw - exp jest funkcją rosnącą
print('overall accuracy: %.2f' % (np.mean(predicted_class == Y)))

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=predicted_class, s=40, cmap=plt.cm.Spectral)
plt.show()

Jak widać, klasyfikator liniowy nie podołał zadaniu - co nie jest zaskoczeniem. Zobaczmy, jak sytuacja się zmieni po dodaniu jeszcze jednej warstwy.

In [ ]:
# randomizacja wartości początkowych
D = 2
K = 3
h = 12 # size of hidden layer
W = np.random.randn(D,h) 
#W = np.zeros((D,h)) 
b = np.zeros((1,h))
W2 = np.random.randn(h,K) 
#W2 = np.zeros((h,K)) 
b2 = np.zeros((1,K))

# parametry 
step_size = 1e-0
reg = 5e-2 # regularization strength


num_examples = X.shape[0]
for i in range(30):  
    # obliczamy scory oraz przynależności do klas przy aktualnych wagach
    hidden_layer = np.maximum(0, np.dot(X, W) + b) # warstwa ukryta - stosujemy funkcję aktywacyjną ReLU
    scores = np.dot(hidden_layer, W2) + b2 # warstwa końcowa

    exp_scores = np.exp(scores)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True) # [N x K]

    # obliczamy funkcję straty dla aktualnej predykcji
    corect_logprobs = -np.log(probs[range(num_examples),Y])
    data_loss = np.sum(corect_logprobs)/num_examples
    reg_loss = 0.5*reg*np.sum(W*W) + 0.5 * reg * np.sum(W2*W2)
    loss = data_loss + reg_loss
    if i % 10 == 0:
        print("iteration %d: loss %f" % (i, loss))

    # wyliczamy gradient funkcji straty
    dscores = probs
    dscores[range(num_examples),Y] -= 1
    dscores /= num_examples

    # wsteczna propagacja gradientu
    # najpierw badamy wpływ wag i stałej w ostatniej warstwie
    dW2 = np.dot(hidden_layer.T, dscores)
    db2 = np.sum(dscores, axis=0, keepdims=True)
    
    # następnie liczymy gradient dla wartości w warstwie ukrytej
    dhidden = np.dot(dscores, W2.T)
    # pamietamy o uwzględnieniu pochodnej funkcji aktywacyjnej - na szczęście jest dość prosta
    dhidden[hidden_layer <= 0] = 0
    # na koniec dostajemy gradienty dla pierwszej warstwy
    dW = np.dot(X.T, dhidden)
    db = np.sum(dhidden, axis=0, keepdims=True)

    # dodajemy jeszcze wpływ regularyzacji do gradientu wag
    dW2 += reg * W2
    dW += reg * W

    # aktualizujemy parametry
    W += -step_size * dW
    b += -step_size * db
    W2 += -step_size * dW2
    b2 += -step_size * db2

In [ ]:
hidden_layer = np.maximum(0, np.dot(X, W) + b)
scores = np.dot(hidden_layer, W2) + b2
predicted_class = np.argmax(scores, axis=1)
print('training accuracy: %.2f' % (np.mean(predicted_class == Y)))

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=predicted_class, s=40, cmap=plt.cm.Spectral)
plt.show()

Uf, dużo lepiej.

## 2. Keras

In [ ]:
import keras

Podstawowe moduły:  
- models - tutaj główną klasą jest Sequential, i z tej klasy będziemy korzystać najcześciej
- layers - różne typy warstw, m.in.:
    - Dense - warstwa z wszystkimi połączeniami
    - Convolutional - warstwa konwolucyjna
    - Activation - nałożenie na poprzednią warstwę funkcji aktywacyjnej
- optimizers - algorytmy do szukania optymalnych wartości wag, m.in.:
    - SGD - metoda gradientu prostego
    - Adam, RMSProp - bardziej zaawansowane metody korzystające ze zmiennego tempa uczenia
- utils - przydatne narzędzia do prostej obróbki danych, m.in normalizacja czy kodowanie kategorycznej zmiennej objaśnianej

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD, adam
from keras.utils.np_utils import to_categorical

#### 2.1 Kręgi w zbożu

Najpierw zbudujemy sieci analogiczne do poprzednich - dzięki temu zobaczymy jak działają podstawowe funkcje i klasy, oraz przekonamy się jak bardzo user-friendly jest keras ;)

Na pierwszy ogień - model liniowy.


Najpierw budujemy pusty model typu `Sequential()`. Następnie możemy dodać do niego kolejne warstwy wywołując na tym obiekcie metodę `add()`

In [ ]:
model_linear = Sequential()
model_linear.add(Dense(3, input_dim = 2)) # dla pierwszej warstwy musimy podać ilosć neuronów na wejściu - 
                                   # dla pozostałych nie musimy się martwić
model_linear.add(Activation('softmax'))

Po zbudowaniu modelu, musimy go skompilować. Używamy metody `compile`, podając algorytm optymalizacji oraz funkcje straty, którą będziemy chcieli optymalizować.

In [ ]:
model_linear.compile(optimizer=SGD(lr = 1),
              loss='categorical_crossentropy', metrics=['accuracy']) # parametry jak wcześniej

Parametr `batch_size` określa ile obserwacji będziemy używać w jednym kroku aktualizacji wag. Jest to szczególnie przydatne wtedy, jeśli mamy bardzo dużo danych i komputer nie wyrabia z mnożeniem macierzy takiej wielkości.  
W tym przypadku możemy spokojnie ustawić `batch_size=600`, aby zachować analogię do pierwszych przykładów.

In [ ]:
# uczymy! 
model_linear.fit(X, to_categorical(Y), epochs=20, verbose = 1, batch_size=600) 

In [ ]:
preds_linear = model_linear.predict(X)
print('training accuracy: %.2f' % (np.mean(np.argmax(preds_linear, axis=1) == Y)))

Następnie model z jedną warstwą ukrytą. Postępujemy tak, jak wcześniej, ale tym razem dodajemy o jedną warstwę więcej.

In [ ]:
model_2l = Sequential()
model_2l.add(Dense(12, input_dim=2))
model_2l.add(Activation('relu'))
model_2l.add(Dense(3))
model_2l.add(Activation('softmax'))

model_2l.compile(optimizer=SGD(lr = 1),
              loss='categorical_crossentropy')

# uczymy! 
model_2l.fit(X, to_categorical(Y), epochs=20, verbose = 1, batch_size=600) 

In [ ]:
preds_2l = model_2l.predict(X)
print('training accuracy: %.2f' % (np.mean(np.argmax(preds_2l, axis=1) == Y)))

### Bostońskie domostwa

In [ ]:
from keras.datasets import boston_housing 

1. CRIM: per capita crime rate by town  
2. ZN: proportion of residential land zoned for lots over 25,000 sq.ft. 
3. INDUS: proportion of non-retail business acres per town 
4. CHAS: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise) 
5. NOX: nitric oxides concentration (parts per 10 million) 
6. RM: average number of rooms per dwelling 
7. AGE: proportion of owner-occupied units built prior to 1940 
8. DIS: weighted distances to five Boston employment centres 
9. RAD: index of accessibility to radial highways 
10. TAX: full-value property-tax rate per \$10,000  
11. PTRATIO: pupil-teacher ratio by town  
12. B: $1000(B_k - 0.63)^2$ where Bk is the proportion of blacks by town 
13. LSTAT:  lower status of the population 
14. MEDV: Median value of owner-occupied homes in 1000's

In [ ]:
(X_train_boston, Y_train_boston), (X_test_boston, Y_test_boston) = boston_housing.load_data()
print(X_train_boston.shape)

In [ ]:
model_boston_linear = Sequential()
model_boston_linear.add(Dense(1, input_dim=13))
model_boston_linear.add(Activation('linear'))

model_boston_linear.compile(optimizer=SGD(), loss="mean_squared_error")

model_boston_linear.fit(X_train_boston, Y_train_boston, epochs=3, batch_size=404, verbose = 1)

In [ ]:
model_boston_linear.get_weights()

Wszystkie wagi wybuchają - trzeba coś z tym zrobić

In [ ]:
def describe_column(col):
    return( (np.min(col), np.max(col), np.mean(col)))

In [ ]:
stats = np.apply_along_axis(describe_column, 0, X_train_boston)
stats.T

Możemy podzielić każdą kolumnę przez jej maksimum

In [ ]:
# podzielmy przez maximum dla każdej kolumny
X_train_boston /= stats[1]
X_test_boston /= stats[1]

In [ ]:
model_boston_linear = Sequential()
model_boston_linear.add(Dense(1, input_dim=13))
model_boston_linear.add(Activation('linear'))

model_boston_linear.compile(optimizer="sgd", loss="mean_squared_error")

model_boston_linear.fit(X_train_boston, Y_train_boston, epochs=50, batch_size=404, verbose=1)

In [ ]:
np.mean((model_boston_linear.predict(X_test_boston).reshape(Y_test_boston.shape) - Y_test_boston)**2)

Zobaczmy czy dodanie kolejnej warstwy poprawi wynik - zbuduj model z jedną warstwą ukrytą.

### Rozpoznawanie pisma

In [ ]:
from keras.datasets import mnist

In [ ]:
(X_train_mnist, Y_train_mnist), (X_test_mnist, Y_test_mnist) = mnist.load_data()
print(X_train_mnist.shape)
print(Y_train_mnist.shape)

In [ ]:
digit = X_train_mnist[2]
plt.imshow(digit, interpolation = "nearest", cmap = "gray")
plt.show()

Do wypróbowania:
    - subsampling obrazka 
        from scipy import ndimage
        test_x2 = np.array([ndimage.interpolation.zoom(i, 18/28) for i in test_x])
    - normalizacja 
    - binaryzacja
        X_train_mnist[X_train_mnist<cutoff] = 0
    - różne rodzaje funkcji aktywacyjnej